## 04 - Scraping party votes

In [16]:
import pandas as pd
import re
import numpy as np

df = pd.read_csv("votes.csv", na_values=['NaN'])

df['confirmation_date'] = pd.to_datetime(df.confirmation_date)

# Convert the votes and record vote numbers into integers instead of float to remove the decimals. 
df['yea_votes'] = df['yea_votes'].astype('Int64')
df['nay_votes'] = df['nay_votes'].astype('Int64')
df['record_vote_number'] = df['record_vote_number'].astype('Int64')

In [17]:
# Convert the record_vote_number into a three digit string – this will be important later when I have to loop
# through URLs.
df['record_vote_number'] = df['record_vote_number'].apply(lambda x: '{0:0>3}'.format(x))
df.head(20)

,name,nomination_no,congress_no,circuit,confirmation_date,yea_votes,nay_votes,record_vote_number
0,Roopali H. Desai,PN2262,117,Ninth,2022-08-04,67,29,284
1,Florence Y. Pan,PN2193,117,District of Columbia,2022-09-20,52,42,340
2,Sarah A. L. Merriam,PN2141,117,Second,2022-09-15,53,44,337
3,Lara E. Montecalvo,PN2140,117,First,2022-09-14,52,47,335
4,Salvador Mendoza Jr,PN1966,117,Ninth,2022-09-12,46,40,331
5,John Z. Lee,PN1965,117,Seventh,2022-09-07,50,44,327
6,Stephanie Dawkins Davis,PN1748,117,Sixth,2022-05-24,49,43,194
7,Julianna Michelle Childs,PN1671,117,District of Columbia,2022-07-19,64,34,260
8,Leonard Philip Stark,PN1508,117,Federal,2022-02-09,61,35,049
9,Alison J. Nathan,PN1504,117,Second,2022-03-23,49,47,106


In [18]:
df['confirmation_year']=df['confirmation_date'].dt.to_period('Y')

df.confirmation_year = df.confirmation_year.map(str)
df.confirmation_year = df.confirmation_year.astype(int)

df['session'] = df.confirmation_year.apply(lambda x: "2" if x % 2 == 0 else "1")
df.head(20)

,name,nomination_no,congress_no,circuit,confirmation_date,yea_votes,nay_votes,record_vote_number,confirmation_year,session
0,Roopali H. Desai,PN2262,117,Ninth,2022-08-04,67,29,284,2022,2
1,Florence Y. Pan,PN2193,117,District of Columbia,2022-09-20,52,42,340,2022,2
2,Sarah A. L. Merriam,PN2141,117,Second,2022-09-15,53,44,337,2022,2
3,Lara E. Montecalvo,PN2140,117,First,2022-09-14,52,47,335,2022,2
4,Salvador Mendoza Jr,PN1966,117,Ninth,2022-09-12,46,40,331,2022,2
5,John Z. Lee,PN1965,117,Seventh,2022-09-07,50,44,327,2022,2
6,Stephanie Dawkins Davis,PN1748,117,Sixth,2022-05-24,49,43,194,2022,2
7,Julianna Michelle Childs,PN1671,117,District of Columbia,2022-07-19,64,34,260,2022,2
8,Leonard Philip Stark,PN1508,117,Federal,2022-02-09,61,35,049,2022,2
9,Alison J. Nathan,PN1504,117,Second,2022-03-23,49,47,106,2022,2


In [19]:
# Because the record vote numbers are repeated every year, some nominations share the number. Later on I will need a unique number for each nomination,
# so I create a vote_id by merging the congress #, the session # and and the record vote #
df['vote_id'] = df.congress_no.astype(str) + df.session.astype(str) + df.record_vote_number.astype(str)
df

,name,nomination_no,congress_no,circuit,confirmation_date,yea_votes,nay_votes,record_vote_number,confirmation_year,session,vote_id
0,Roopali H. Desai,PN2262,117,Ninth,2022-08-04,67,29,284,2022,2,1172284
1,Florence Y. Pan,PN2193,117,District of Columbia,2022-09-20,52,42,340,2022,2,1172340
2,Sarah A. L. Merriam,PN2141,117,Second,2022-09-15,53,44,337,2022,2,1172337
3,Lara E. Montecalvo,PN2140,117,First,2022-09-14,52,47,335,2022,2,1172335
4,Salvador Mendoza Jr,PN1966,117,Ninth,2022-09-12,46,40,331,2022,2,1172331
...,...,...,...,...,...,...,...,...,...,...,...
189,Terrence L. O'Brien,PN882,107,Tenth,2002-04-15,98,0,068,2002,2,1072068
190,Michael J. Melloy,PN881,107,Eighth,2002-02-11,91,0,021,2002,2,1072021
191,Michael W. McConnell,PN880,107,Tenth,2002-11-15,<NA>,<NA>,<NA>,2002,2,1072<NA>
192,William J. Riley,PN458,107,Eighth,2001-08-02,97,0,270,2001,1,1071270


In [7]:
from bs4 import BeautifulSoup
import requests
from playwright.async_api import async_playwright

playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

In [9]:
# DO NOT RUN ALL THE TIME! MIGHT UPSET THE SERVER...

votes = [] 
for index, row in df.iterrows():
    record_vote_number = row['record_vote_number']
    congress_no = row['congress_no']
    session = row['session']
    await page.goto(f"https://www.senate.gov/legislative/LIS/roll_call_votes/vote{congress_no}{session}/vote_{congress_no}_{session}_00{record_vote_number}.htm")

    html = await page.content()

    doc = BeautifulSoup(html)

    nominations = {}
    nominations["congress_no"] = row['congress_no']
    nominations["session"] = row['session']
    nominations["vote_number"] = doc.select("#secondary_col2 > div:nth-child(7) > div:nth-child(2)")

    results = doc.select("#secondary_col2 > div:nth-child(14) > span ")

    nomination = []
    for row in results:
        nomination.append(row.text)

    nomination1 = [item.split('\n') for item in nomination]

    nomination1_flat = [item for l in nomination1 for item in l]

    nomination2 = [item.split(',') for item in nomination1_flat]

    nominations["votes"] = nomination2

    votes.append(nominations)

In [21]:
import pandas as pd
df = pd.DataFrame(votes)

df.to_csv("party_votes.csv", index=False)

[{'congress_no': 117,
  'session': '2',
  'vote_number': [<div class="contenttext" style="float:left; min-width:200px; padding-bottom:10px;">
   <b>Vote Number:   </b>284</div>],
  'votes': [['Baldwin (D-WI)', ' Yea'],
   ['Barrasso (R-WY)', ' Nay'],
   ['Bennet (D-CO)', ' Yea'],
   ['Blackburn (R-TN)', ' Nay'],
   ['Blumenthal (D-CT)', ' Yea'],
   ['Blunt (R-MO)', ' Nay'],
   ['Booker (D-NJ)', ' Yea'],
   ['Boozman (R-AR)', ' Nay'],
   ['Braun (R-IN)', ' Nay'],
   ['Brown (D-OH)', ' Yea'],
   ['Burr (R-NC)', ' Not Voting'],
   ['Cantwell (D-WA)', ' Yea'],
   ['Capito (R-WV)', ' Yea'],
   ['Cardin (D-MD)', ' Yea'],
   ['Carper (D-DE)', ' Yea'],
   ['Casey (D-PA)', ' Yea'],
   ['Cassidy (R-LA)', ' Yea'],
   ['Collins (R-ME)', ' Yea'],
   ['Coons (D-DE)', ' Yea'],
   ['Cornyn (R-TX)', ' Not Voting'],
   ['Cortez Masto (D-NV)', ' Yea'],
   ['Cotton (R-AR)', ' Nay'],
   ['Cramer (R-ND)', ' Yea'],
   ['Crapo (R-ID)', ' Yea'],
   ['Cruz (R-TX)', ' Nay'],
   ['Daines (R-MT)', ' Nay'],
   ['Du

In [30]:
for vote in votes:
    vote = {}
    vote['D_yea'] = 0
    vote['D_nay'] = 0
    vote['D_no_vote'] = 0
    vote['R_yea'] = 0
    vote['R_nay'] = 0
    vote['R_no_vote'] = 0

    for senator in vote['votes']:
        print(senator)
        print("------")
        if senator[0].contains("(D"):
            if senator[1] == " Yea":

['Baldwin (D-WI)', ' Yea']
------
['Barrasso (R-WY)', ' Nay']
------
['Bennet (D-CO)', ' Yea']
------
['Blackburn (R-TN)', ' Nay']
------
['Blumenthal (D-CT)', ' Yea']
------
['Blunt (R-MO)', ' Nay']
------
['Booker (D-NJ)', ' Yea']
------
['Boozman (R-AR)', ' Nay']
------
['Braun (R-IN)', ' Nay']
------
['Brown (D-OH)', ' Yea']
------
['Burr (R-NC)', ' Not Voting']
------
['Cantwell (D-WA)', ' Yea']
------
['Capito (R-WV)', ' Yea']
------
['Cardin (D-MD)', ' Yea']
------
['Carper (D-DE)', ' Yea']
------
['Casey (D-PA)', ' Yea']
------
['Cassidy (R-LA)', ' Yea']
------
['Collins (R-ME)', ' Yea']
------
['Coons (D-DE)', ' Yea']
------
['Cornyn (R-TX)', ' Not Voting']
------
['Cortez Masto (D-NV)', ' Yea']
------
['Cotton (R-AR)', ' Nay']
------
['Cramer (R-ND)', ' Yea']
------
['Crapo (R-ID)', ' Yea']
------
['Cruz (R-TX)', ' Nay']
------
['Daines (R-MT)', ' Nay']
------
['Duckworth (D-IL)', ' Yea']
------
['Durbin (D-IL)', ' Yea']
------
['Ernst (R-IA)', ' Yea']
------
['Feinstein (D-C